In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [2]:
train_data = pd.read_csv('/home/neeraj/python_programs/kaggle_practice/twitter_disaster/train.csv')
test_data = pd.read_csv('/home/neeraj/python_programs/kaggle_practice/twitter_disaster/test.csv')

In [17]:
train_data.head()
test_data.head()

,id,keyword,location,text,length
0,0,NaN,NaN,Just happened a terrible car crash,34
1,2,NaN,NaN,"Heard about #earthquake is different cities, s...",64
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are...",96
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires,40
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan,45


In [4]:
test_data['length'] = test_data['text'].apply(len)
test_data.head()

,id,keyword,location,text,length
0,0,NaN,NaN,Just happened a terrible car crash,34
1,2,NaN,NaN,"Heard about #earthquake is different cities, s...",64
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are...",96
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires,40
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan,45


In [5]:
import string

mess = 'Sample message! Notice: it has punctuation.'

# Check characters to see if they are in punctuation
nopunc = [char for char in mess if char not in string.punctuation]

# Join the characters again to form the string.
nopunc = ''.join(nopunc)

In [6]:
from nltk.corpus import stopwords
stopwords.words('english')[0:10] # Show some stop words
nopunc.split()

['Sample', 'message', 'Notice', 'it', 'has', 'punctuation']

In [7]:
def text_process(mess):
    """
    Takes in a string of text, then performs the following:
    1. Remove all punctuation
    2. Remove all stopwords
    3. Returns a list of the cleaned text
    """
    # Check characters to see if they are in punctuation
    nopunc = [char for char in mess if char not in string.punctuation]

    # Join the characters again to form the string.
    nopunc = ''.join(nopunc)
    
    # Now just remove any stopwords
    return [word for word in nopunc.split() if word.lower() not in stopwords.words('english')]

In [8]:
train_data['text'].head(5).apply(text_process)

0    [Deeds, Reason, earthquake, May, ALLAH, Forgiv...
1        [Forest, fire, near, La, Ronge, Sask, Canada]
2    [residents, asked, shelter, place, notified, o...
3    [13000, people, receive, wildfires, evacuation...
4    [got, sent, photo, Ruby, Alaska, smoke, wildfi...
Name: text, dtype: object

In [9]:
from sklearn.feature_extraction.text import CountVectorizer
bow_transformer = CountVectorizer(analyzer=text_process).fit(train_data['text'])

# Print total number of vocab words
print(len(bow_transformer.vocabulary_))



26473


In [10]:
train_bow = bow_transformer.transform(train_data['text'])
test_bow = bow_transformer.transform(test_data['text'])

In [11]:
print('Shape of Sparse Matrix: ', train_bow.shape)

Shape of Sparse Matrix:  (7613, 26473)


In [12]:
from sklearn.feature_extraction.text import TfidfTransformer

tfidf_transformer = TfidfTransformer().fit(train_bow)
train_tfidf = tfidf_transformer.transform(train_bow)
test_tfidf = tfidf_transformer.transform(test_bow)

In [13]:
from sklearn.naive_bayes import MultinomialNB
tweet_detect_model = MultinomialNB().fit(train_tfidf, train_data['target'])

In [30]:
predictions = tweet_detect_model.predict(test_tfidf)

In [31]:
print(predictions)

[1 1 1 ... 1 1 1]


In [32]:
my_submission = pd.DataFrame({'Id': test_data['id'], 'target': predictions})

In [34]:
my_submission.to_csv('submission.csv', index=False)